In [1]:
%load_ext line_profiler
%load_ext memory_profiler

# Day 2

## Exercise 3.a

To run line profiler on our code, we can define the script as a function and then run %lprun to see how much time is spent execting each line.

In [19]:
# Program to multiply two matrices using nested loops
import random

def matmult():
    N = 250

    # NxN matrix
    X = []
    for i in range(N):
        X.append([random.randint(0,100) for r in range(N)])

    # Nx(N+1) matrix
    Y = []
    for i in range(N):
        Y.append([random.randint(0,100) for r in range(N+1)])

    # result is Nx(N+1)
    result = []
    for i in range(N):
        result.append([0] * (N+1))

    # iterate through rows of X
    for i in range(len(X)):
        # iterate through columns of Y
        for j in range(len(Y[0])):
            # iterate through rows of Y
            for k in range(len(Y)):
                result[i][j] += X[i][k] * Y[k][j]

    #for r in result:
    #    print(r)

Now let's run `lprun` to check the stats.

In [20]:
%lprun -T mprof0 -f matmult matmult()
print(open('mprof0', 'r').read())


*** Profile printout saved to text file 'mprof0'. 
Timer unit: 1e-06 s

Total time: 22.3254 s
File: <ipython-input-19-bf771f301bd1>
Function: matmult at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def matmult():
     5         1          2.0      2.0      0.0      N = 250
     6                                           
     7                                               # NxN matrix
     8         1          1.0      1.0      0.0      X = []
     9       251        232.0      0.9      0.0      for i in range(N):
    10       250     208846.0    835.4      0.9          X.append([random.randint(0,100) for r in range(N)])
    11                                           
    12                                               # Nx(N+1) matrix
    13         1          1.0      1.0      0.0      Y = []
    14       251        156.0      0.6      0.0      for i in range(N):
    15       250     178409.0    713.6   

In the output table, we can see that most of the time is spent on executing lines 27-28:

`Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================`

`15750250    8709942.0      0.6     35.0              for k in range(len(Y)):`

`15687500   15686312.0      1.0     63.1                  result[i][j] += X[i][k] * Y[k]`

Now let's look at memory usage with memory profiler

In [14]:
%memit test_matmult()

peak memory: 54.51 MiB, increment: 0.87 MiB


We can also go line-by-line using `%mprun`, but this actually took so long time to run so I've skipped it. Maybe this has to do with the nested for loops? :-/

### Answer to exercise 3.a

The lines of code using the most time were 27-28, the innermost for loop. As for memory, the `result` variable is initiated before the nested for loop, so the for loop in itself shpuld not be using that mush memory. The majority of memory is used when the function is initiated.

## Exercise 3.c

Now let's try to optimize the code. Instead of using the default python lists, we can define the matrices as numpy arrays instead. 

If we use `np.random.randint` we can set the shape of the array directly, so we don't have to make a for loop. (I have added a random seed before initiating the randomized matrices X and Y to be able to compare the results with the matmult function from numpy)

In the for loop, we iterate through rows of `X` and columns of `Y`. In the original code, used another for loop to calculate the product of `X[i][k]` and `Y[k][j]`. Instead, we can use vector multiplication to get the product of `X[i]*Y[i]` and sum all elements of the product vector. Is cheating? :-/

In [15]:
# Program to multiply two matrices using nested loops
import numpy as np

def matmult():
    N = 250

    # NxN matrix
    np.random.seed(0)
    X = np.random.randint(0,100, size=(N, N))

    # Nx(N+1) matrix
    np.random.seed(0)
    Y = np.random.randint(0,100, size=(N, N + 1))

    # Nx(N+1)
    result = np.zeros(shape=(N, N+1), dtype="int64")

    # iterate through rows of X
    for i in range(X.shape[0]):
        # iterate through columns of Y
        for j in range(Y.shape[1]):
            a = X[i, :]*Y[:, j]
            result[i, j] = a.sum()
            
    #for r in result:
    #print(r)
    
    return result

Again, let's run line profiler on the code.

In [27]:
%time matmult()
%lprun -T mprof0 -f matmult matmult()
print(open('mprof0', 'r').read())

CPU times: user 312 ms, sys: 3.2 ms, total: 315 ms
Wall time: 315 ms

*** Profile printout saved to text file 'mprof0'. 
Timer unit: 1e-06 s

Total time: 0.453911 s
File: /Users/ludviglarsson/courses/uu-python-course/exercises/uu-exercise-Ludvig/day2/matmult_optimized.py
Function: matmult at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def matmult():
     6         1          1.0      1.0      0.0      N = 250
     7                                           
     8                                               # NxN matrix
     9         1         10.0     10.0      0.0      np.random.seed(0)
    10         1        502.0    502.0      0.1      X = np.random.randint(0,100, size=(N, N))
    11                                           
    12                                               # Nx(N+1) matrix
    13         1          5.0      5.0      0.0      np.random.seed(0)
    14         1        494.0    494.0

Now we can see that our new code made a significant improvement to the computation time.

`Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================`

`    22     62750     193281.0      3.1     40.7              a = X[i, :]*Y[:, j]`

`    23     62750     251702.0      4.0     53.0              result[i, j] = a.sum()`


In [21]:
%%file matmult_optimized.py

# Program to multiply two matrices using nested loops
import numpy as np

def matmult():
    N = 250

    # NxN matrix
    np.random.seed(0)
    X = np.random.randint(0,100, size=(N, N))

    # Nx(N+1) matrix
    np.random.seed(0)
    Y = np.random.randint(0,100, size=(N, N + 1))

    # Nx(N+1)
    result = np.zeros(shape=(N, N+1), dtype="int64")

    # iterate through rows of X
    for i in range(X.shape[0]):
        # iterate through columns of Y
        for j in range(Y.shape[1]):
            a = X[i, :]*Y[:, j]
            result[i, j] = a.sum()
            
    #for r in result:
    #print(r)
    
    return result

Overwriting matmult_optimized.py


In [24]:
from matmult_optimized import matmult
%memit matmult()
%mprun -T mprof0 -f matmult matmult()
print(open('mprof0', 'r').read())

peak memory: 70.85 MiB, increment: 0.00 MiB


*** Profile printout saved to text file mprof0. 
Filename: /Users/ludviglarsson/courses/uu-python-course/exercises/uu-exercise-Ludvig/day2/matmult_optimized.py

Line #    Mem usage    Increment  Occurences   Line Contents
     5     70.8 MiB     70.8 MiB           1   def matmult():
     6     70.8 MiB      0.0 MiB           1       N = 250
     7                                         
     8                                             # NxN matrix
     9     70.8 MiB      0.0 MiB           1       np.random.seed(0)
    10     70.8 MiB      0.0 MiB           1       X = np.random.randint(0,100, size=(N, N))
    11                                         
    12                                             # Nx(N+1) matrix
    13     70.8 MiB      0.0 MiB           1       np.random.seed(0)
    14     70.8 MiB      0.0 MiB           1       Y = np.random.randint(0,100, size=(N, N + 1))
    15                                         
    16 

We can also check if our matrix multiplication method generates the same result as `np.matmul`.

In [18]:
N = 250

# NxN matrix
np.random.seed(0)
X = np.random.randint(0,100, size=(N, N))

np.random.seed(0)
Y = np.random.randint(0,100, size=(N, N + 1))

# Result with numpy matmul
A1 = np.matmul(X, Y)

# Result with custom matmult
A2 = matmult()

np.array_equal(A1, A2)

True

### Answer to exercise 3.c

The best performance at N=250 was: 

* CPU times: user 312 ms, sys: 3.2 ms, total: 315 ms

* Wall time: 315 ms

* peak memory: 70.85 MiB, increment: 0.00 MiB